<a href="https://colab.research.google.com/github/AaronJing/FreezeMLP/blob/mnist-fashion/mlpfreezing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-e9684b01-8aa3-0a54-1ac7-43945bc113f9)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
batch_size = 64

In [ ]:
# Prepare data set
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()



In [ ]:

x_train = x_train / 255

x_test = x_test / 255

In [ ]:

class model():
  def __init__(self,frez=False):
    def dense(name, neurons, kernel_initializer='glorot_uniform',activation='relu'):
      return layers.Dense(neurons, activation = activation, name = name, kernel_initializer = kernel_initializer)
    inputs = keras.Input(shape=(28,28),name = "InputLayer")
    x = layers.Flatten()(inputs)
    i = 0 
    self.freezed = frez
    while i < 7:
      if i in [1,3,5]:
        x = dense('dense'+str(i),192)(x)
      else:
        if self.freezed:
          x = dense('dense'+str(i),192, tf.keras.initializers.Identity())(x)
        else:
          x = dense('dense'+str(i),192)(x)
      i += 1
    x = layers.Dense(10, name='dense_logits')(x)
    outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
    self.model = keras.Model(inputs=inputs, outputs=outputs)
    self.compile()

  def compile(self,lr=0.001):
    self.model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr),
    metrics=['accuracy'])
  def fit(self,train_set, valid_set):
    self.model.fit(train_set,
                    validation_data = valid_set,
                    epochs=5)

  def fit(self,x,y,epoch,batch_size,callbacks=None):
    # fit(
#     x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None,
#     validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
#     sample_weight=None, initial_epoch=0, steps_per_epoch=None,
#     validation_steps=None, validation_batch_size=None, validation_freq=1,
#     max_queue_size=10, workers=1, use_multiprocessing=False
# )
    self.model.fit(x=x,y=y,batch_size=batch_size, epochs = epoch, callbacks = callbacks,validation_split= 0.05)
  def evaluate(self,test_set):
    return self.model.evaluate(test_set, verbose=2)
  def evaluate(self,x_test, y_test):
    return self.model.evaluate(x_test,y_test, verbose=2)
  def get_weights(self):
    i = 0 
    while i < 5:
      print('dense'+str(i))
      print(self.model.get_layer('dense'+str(i)).weights)
      i += 1
  def summary(self):
    return self.model.summary()
  def freeze(self,iteration):

    if iteration == 1:
      i = 0
      while i < 7:
        l = self.model.get_layer('dense'+str(i))
        if i not in [1,3,5]:
          l.trainable = False
          print('dense'+str(i)+" freezed")
        else:
          l.trainable = True
        i += 1
        self.compile(0.001)
    elif iteration == 2 :
      i = 0
      while i < 7:
        l = self.model.get_layer('dense'+str(i))
        if i not in [0,2,4,6]:
          l.trainable = False
          print('dense'+str(i)+" freezed")
        else:
          l.trainable = True
        i += 1
      self.compile(0.0005)
    else:
      i = 0
      while i < 7:
        l = self.model.get_layer('dense'+str(i))
        l.trainable = True
        i+=1
      self.compile(0.0005)
        
# construct model and initial weight
model_frez = model(True)


In [ ]:
import os
model_frez.freeze(1)
model_frez.summary()
# print(model.get_weights())
history = model_frez.fit(x_train,y_train,5,64)
test_scores = model_frez.evaluate(x_test, y_test)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])
model_frez.summary()


dense0 freezed
dense2 freezed
dense4 freezed
dense6 freezed
Model: "model_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      [(None, 28, 28)]          0         
_________________________________________________________________
flatten_42 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense0 (Dense)               (None, 192)               150720    
_________________________________________________________________
dense1 (Dense)               (None, 192)               37056     
_________________________________________________________________
dense2 (Dense)               (None, 192)               37056     
_________________________________________________________________
dense3 (Dense)               (None, 192)               37056     
________________________________________________________________

In [ ]:
model_frez.freeze(2)
model_frez.evaluate(x_test,y_test)
model_frez.fit(x_train,y_train,5,64)
model_frez.evaluate(x_test,y_test)

dense1 freezed
dense3 freezed
dense5 freezed
313/313 - 1s - loss: 0.7282 - accuracy: 0.7101
Epoch 1/5
891/891 [==============================] - 3s 3ms/step - loss: 0.4249 - accuracy: 0.8414 - val_loss: 0.3472 - val_accuracy: 0.8667
Epoch 2/5
891/891 [==============================] - 3s 3ms/step - loss: 0.3440 - accuracy: 0.8721 - val_loss: 0.3216 - val_accuracy: 0.8823
Epoch 3/5
891/891 [==============================] - 3s 3ms/step - loss: 0.3163 - accuracy: 0.8815 - val_loss: 0.3231 - val_accuracy: 0.8777
Epoch 4/5
891/891 [==============================] - 3s 3ms/step - loss: 0.3015 - accuracy: 0.8875 - val_loss: 0.3303 - val_accuracy: 0.8767
Epoch 5/5
891/891 [==============================] - 3s 3ms/step - loss: 0.2870 - accuracy: 0.8922 - val_loss: 0.3169 - val_accuracy: 0.8837
313/313 - 0s - loss: 0.3491 - accuracy: 0.8805


[0.3491112291812897, 0.8805000185966492]

In [ ]:
model_once_train = model(False)
def scheduler(epoch, lr):
  if epoch > 5:
    return 0.0005
  else:
    return 0.001
model_once_train.fit(x_train,y_train,10,64,[ tf.keras.callbacks.LearningRateScheduler(scheduler)])
model_once_train.evaluate(x_test,y_test)

Epoch 1/10
891/891 [==============================] - 3s 3ms/step - loss: 0.7227 - accuracy: 0.7340 - val_loss: 0.4172 - val_accuracy: 0.8420
Epoch 2/10
891/891 [==============================] - 3s 3ms/step - loss: 0.4039 - accuracy: 0.8545 - val_loss: 0.4081 - val_accuracy: 0.8433
Epoch 3/10
891/891 [==============================] - 3s 3ms/step - loss: 0.3569 - accuracy: 0.8687 - val_loss: 0.3466 - val_accuracy: 0.8713
Epoch 4/10
891/891 [==============================] - 3s 3ms/step - loss: 0.3299 - accuracy: 0.8824 - val_loss: 0.3536 - val_accuracy: 0.8783
Epoch 5/10
891/891 [==============================] - 3s 3ms/step - loss: 0.3099 - accuracy: 0.8858 - val_loss: 0.3604 - val_accuracy: 0.8710
Epoch 6/10
891/891 [==============================] - 3s 3ms/step - loss: 0.2961 - accuracy: 0.8911 - val_loss: 0.3553 - val_accuracy: 0.8767
Epoch 7/10
891/891 [==============================] - 3s 3ms/step - loss: 0.2604 - accuracy: 0.9030 - val_loss: 0.3170 - val_accuracy: 0.8850
Epoch 

[0.34073713421821594, 0.8812999725341797]